### Required Libraries

In [ ]:
# !pip install -qU langchain tiktoken matplotlib seaborn tqdm
# !pip install pyenchant   ### for misspling 

### Loading the Data

In [8]:
import json
import pandas as pd
import os

# Step 1: Load JSON Data
file_path = r"C:\A\00Master\Demo\data acc\att\output\DIN 1045-1000.json"# Replace with your file path
with open(file_path, 'r') as file:
    data = json.load(file)

def extract_file_name_without_extension(file_path):
    # Extract the file name from the file path
    file_name_with_extension = os.path.basename(file_path)
    # Split the file name and the extension
    file_name_without_extension, _ = os.path.splitext(file_name_with_extension)
    return file_name_without_extension

filename = extract_file_name_without_extension(file_path)
filename
df = pd.DataFrame(data)
# Removing .png from column names
df.columns = [col.replace('.png', '') for col in df.columns]

filtered_columns = [col for col in df.columns if col.startswith(filename)]

# Extract the first row for these filtered columns
first_row = df[filtered_columns].iloc[0] if filtered_columns else None
df_first_row = pd.DataFrame([first_row])
df_first_row = df_first_row.transpose()
df= df_first_row

In [9]:
!pip install transformers

# Bing Spell Check API 

In [49]:
import requests
import json

api_key = "e5e3afdea1c04de3a80756a6b4691ee3"
example_text = "Hollo, sche%int" # the text to be spell-checked
endpoint = "https://api.bing.microsoft.com/v7.0/SpellCheck"

data = {'text': example_text}

params = {
    'mkt':'en-us',
    'mode':'proof'
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Ocp-Apim-Subscription-Key': api_key,
}

response = requests.post(endpoint, headers=headers, params=params, data=data)

json_response = response.json()
print(json.dumps(json_response, indent=4))


{
    "_type": "SpellCheck",
    "flaggedTokens": [
        {
            "offset": 0,
            "token": "Hollo",
            "type": "UnknownToken",
            "suggestions": [
                {
                    "suggestion": "Hollow",
                    "score": 0.6321395614929775
                }
            ]
        },
        {
            "offset": 7,
            "token": "sche%int",
            "type": "UnknownToken",
            "suggestions": [
                {
                    "suggestion": "chest",
                    "score": 0.6321395614929775
                }
            ]
        }
    ]
}


In [44]:
import pandas as pd
import requests

# Define your Bing Spell Check API key
api_key = "492eb91e16ed4e6d9233bac77cdb4364"

# Define the API endpoint
endpoint = "https://api.bing.microsoft.com/v7.0/SpellCheck"

# Define a function to perform spelling correction using Bing Spell Check API
def correct_spelling_with_api(text):
    data = {'text': text}
    params = {
        'mkt': 'de-DE',  # Specify the market as English (en-us)
        'mode': 'proof'
    }
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Ocp-Apim-Subscription-Key': api_key,
    }
    
    response = requests.post(endpoint, headers=headers, params=params, data=data)
    json_response = response.json()
    
    # Extract and apply the corrected text
    corrected_text = text
    if 'flaggedTokens' in json_response:
        for token in json_response['flaggedTokens']:
            if 'suggestions' in token:
                for suggestion in token['suggestions']:
                    if suggestion['score'] > 0.5:
                        corrected_text = corrected_text.replace(token['token'], suggestion['suggestion'])
    
    return corrected_text


# Apply the correction function to the DataFrame column
df['corrected_text'] = df['text'].apply(correct_spelling_with_api)

# Print the original and corrected text for each row
for index, row in df.iterrows():
    print(f"Original: {row['text']}")
    print(f"Corrected: {row['corrected_text']}")
    print("-" * 40)


Original: Das it ein Beispieltext.
Corrected: Das it ein Beispieltext.
----------------------------------------
Original: Die Sonne sche%int heute.
Corrected: Die Sonne sche%int heute.
----------------------------------------
Original: Ich habe einen Apfeel.
Corrected: Ich habe einen Apfeel.
----------------------------------------
Original: Wie geht es Ihneen?
Corrected: Wie geht es Ihneen?
----------------------------------------
Original: Deutschhhland issst ein schönes Land.
Corrected: Deutschhhland issst ein schönes Land.
----------------------------------------


In [39]:
import difflib
# Function to compare and highlight differences
def compare_and_highlight(original, corrected):
    differ = difflib.Differ()
    diff = list(differ.compare(original.split(), corrected.split()))
    result = []
    for item in diff:
        if item.startswith(' '):
            result.append(item[2:])
        elif item.startswith('- '):
            result.append(f"\033[91m{item[2:]}\033[0m")  # Red color for removed words
        elif item.startswith('+ '):
            result.append(f"\033[92m{item[2:]}\033[0m")  # Green color for added words
    return ' '.join(result)

# Compare and highlight differences
df['comparison'] = df.apply(lambda row: compare_and_highlight(row['text'], row['corrected_text']), axis=1)

# Print the original, corrected, and highlighted text for each row
for index, row in df.iterrows():
    print(f"Original: {row['text']}")
    print(f"Corrected: {row['corrected_text']}")
    print(f"Highlighted: {row['comparison']}")
    print("-" * 40)

Original: DIN 1045-1000 DIN
ICS 91.010.30; 91.080.40
Tragwerke aus Beton, Stahlbeton und Spannbeton -
Teil 1000: Grundlagen und Betonbauqualitätsklassen (BBQ)
Concrete, reinforced and prestressed concrete structures -
Part 1000: Basic principles and quality classes of concrete construction (BBQ)
Structures en beton, beton arme& et beton precontraint -
Partie 1000: Les principes et les classes de qualite de construction en beton (BBQ)
e
s
3
E
£
Gesamtumfang 24 Seiten
DIN-Normenausschuss Bauwesen (NABau)
© DIN Deutsches Institut für Normung e. V. ist Inhaber aller ausschließlichen Rechte weltweit - alle Rechte
der Verwertung: gleich in welcher Form und welchem Verfahren, sind weltweit DIN vorbehalten. wuewedin.de ITITTD

Corrected: DIN 1045-1000 DIN
ICS 91.010.30; 91.080.40
Tragwerke aus Beton, Stahlbeton und Spannbeton -
Teil 1000: Grundlagen und Betonbauqualitätsklassen (BBQ)
Concrete, reinforced and prestressed concrete structures -
Part 1000: Basic principles and quality classes of c

In [40]:
df.head()

,text,corrected_text,comparison
DIN 1045-1000_page_0,DIN 1045-1000 DIN\nICS 91.010.30; 91.080.40\nT...,DIN 1045-1000 DIN\nICS 91.010.30; 91.080.40\nT...,DIN 1045-1000 DIN ICS 91.010.30; 91.080.40 Tra...
DIN 1045-1000_page_1,Inhalt\nSeite\nVorwort „v2. 20m en n en 3\n1 A...,Inhalt\nSeite\nVorwort „v2. 20m en n en 3\n1 A...,Inhalt Seite Vorwort „v2. 20m en n en 3 1 Anwe...
DIN 1045-1000_page_10,Tabelle 2 — Zuordnung von Anwendungsfällen zu ...,Tabelle 2 — Zuordnung von Anwendungsfällen zu ...,Tabelle 2 — Zuordnung von Anwendungsfällen zu ...
DIN 1045-1000_page_11,Tabelle 2 (fortgesetzt)\n=\ns\n3\nE\nB\n12\n,Tabelle 2 (fortgesetzt)\n=\ns\n3\nE\nB\n12\n,Tabelle 2 (fortgesetzt) = s 3 E B 12
DIN 1045-1000_page_12,Tabelle 2 (fortgesetzt)\n2\n2\n=\nE\nE\n5\n2\n...,Tabelle 2 (fortgesetzt)\n2\n2\n=\nE\nE\n5\n2\n...,Tabelle 2 (fortgesetzt) 2 2 = E E 5 2 & & 4 3 ...


In [41]:
print(df)

                                                                    text  \
DIN 1045-1000_page_0   DIN 1045-1000 DIN\nICS 91.010.30; 91.080.40\nT...   
DIN 1045-1000_page_1   Inhalt\nSeite\nVorwort „v2. 20m en n en 3\n1 A...   
DIN 1045-1000_page_10  Tabelle 2 — Zuordnung von Anwendungsfällen zu ...   
DIN 1045-1000_page_11       Tabelle 2 (fortgesetzt)\n=\ns\n3\nE\nB\n12\n   
DIN 1045-1000_page_12  Tabelle 2 (fortgesetzt)\n2\n2\n=\nE\nE\n5\n2\n...   
DIN 1045-1000_page_13  Tabelle 2 (fortgesetzt)\n\nE1\n&\nB\n=\nE\n©\n...   
DIN 1045-1000_page_14  5 Maßnahmen zur Sicherstellung der Betonbauqua...   
DIN 1045-1000_page_15  5.4 Kenntnisse in der Betonbautechnik\nZur Erf...   
DIN 1045-1000_page_16  AnhangA\n(normativ)\nKommunikation im Rahmen d...   
DIN 1045-1000_page_17  — spezielle Betonanforderungen;\n— ergänzende ...   
DIN 1045-1000_page_18  — Fugenausführung;\n— Anforderungen an die Bet...   
DIN 1045-1000_page_19  (2) Darüber hinaus sollten je nach projektspez...   
DIN 1045-100

In [42]:
df.to_csv("outputspell.csv")

-------------------------------------------------------------------------------------------------

In [ ]:
import requests
import json
api_key = "492eb91e16ed4e6d9233bac77cdb4364"
example_text = "Hollo, wrld" # the text to be spell-checked
endpoint = "https://api.bing.microsoft.com/v7.0/SpellCheck"
data = {'text': example_text}
params = {
    'mkt':'en-us',
    'mode':'proof'
    }
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Ocp-Apim-Subscription-Key': api_key,
    }
response = requests.post(endpoint, headers=headers, params=params, data=data)
json_response = response.json()
print(json.dumps(json_response, indent=4))

In [ ]:
# import pandas as pd
# import requests

# # Define the Bing Spell Check API endpoint and API key
# api_endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/spellcheck"
# api_key = "492eb91e16ed4e6d9233bac77cdb4364"

# # Define a function to perform spelling correction using Bing Spell Check API
# def correct_spelling_with_api(text):
#     headers = {
#         "Ocp-Apim-Subscription-Key": api_key,
#         "Content-Type": "application/x-www-form-urlencoded"
#     }
    
#     params = {
#         "text": text
#     }

#     response = requests.get(api_endpoint, headers=headers, params=params)
#     data = response.json()

#     # Extract the corrected text from the response
#     corrected_text = text
#     if "flaggedTokens" in data:
#         for token in data["flaggedTokens"]:
#             if "suggestions" in token:
#                 for suggestion in token["suggestions"]:
#                     if suggestion["score"] > 0.5:
#                         corrected_text = corrected_text.replace(token["token"], suggestion["suggestion"])
    
#     return corrected_text


# # Initialize a set to store differing words
# differing_words_set = set()

# # Iterate through the DataFrame to find differing words for each row
# for index, row in df.iterrows():
#     original_text = row['text']  # Get the original text from the current row
#     corrected_text = correct_spelling_with_api(original_text)  # Correct the original text

#     # Split the original and corrected text into words
#     original_words = set(original_text.split())
#     corrected_words = set(corrected_text.split())

#     # Find the differing words and update the set
#     differing_words_set.update(original_words.symmetric_difference(corrected_words))

# # Print the collected differing words
# print(f"Differing Words for All Rows: {', '.join(differing_words_set)}")


In [ ]:
import requests

# Define the Bing Spell Check API endpoint and API key
api_endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/spellcheck"
api_key = "492eb91e16ed4e6d9233bac77cdb4364"  # Replace with your actual API key

# Define a function to perform spelling correction using Bing Spell Check API
def correct_spelling_with_api(text):
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Ocp-Apim-Subscription-Key': api_key,
    }
    
    params = {
    'mkt':'en-us',
    'mode':'proof'
    }

    response = requests.get(api_endpoint, headers=headers, params=params, data=)
    data = response.json()

    # Extract the corrected text from the response
    corrected_text = text
    if "flaggedTokens" in data:
        for token in data["flaggedTokens"]:
            if "suggestions" in token:
                for suggestion in token["suggestions"]:
                    if suggestion["score"] > 0.5:
                        corrected_text = corrected_text.replace(token["token"], suggestion["suggestion"])
    
    return corrected_text

# Test the spelling correction function with German text
german_text = "Dit ist ein Beiispiel Satz mit fehlerhaften Wörtern."
corrected_text = correct_spelling_with_api(german_text)

print(f"Original German Text: {german_text}")
print(f"Corrected German Text: {corrected_text}")


In [ ]:
import pandas as pd
import requests

# Define the Bing Spell Check API endpoint and API key
api_endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/spellcheck"
api_key = "492eb91e16ed4e6d9233bac77cdb4364"

# Define a function to perform spelling correction using Bing Spell Check API
def correct_spelling_with_api(text):
    headers = {
        "Ocp-Apim-Subscription-Key": api_key,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    params = {
        "text": text
    }

    response = requests.get(api_endpoint, headers=headers, params=params)
    data = response.json()

    # Extract the corrected text from the response
    corrected_text = text
    if "flaggedTokens" in data:
        for token in data["flaggedTokens"]:
            if "suggestions" in token:
                for suggestion in token["suggestions"]:
                    if suggestion["score"] > 0.5:
                        corrected_text = corrected_text.replace(token["token"], suggestion["suggestion"])
    
    return corrected_text

# Create or load your DataFrame


# Apply the correction function to the DataFrame column
df['text_Correct'] = df['text'].apply(correct_spelling_with_api)
df['is_changed'] = df['text'] != df['text_Correct']
df.head()
# Print the corrected DataFrame
# print(df)


In [ ]:
for  row in df.iloc[1]():
    print(f"Original: {row['text']}")
    print(f"Corrected: {row['text_Correct']}")
    print("-" * 40)

In [ ]:
for index, row in df.iterrows():
    print(f"Original: {row['text']}")
    print(f"Corrected: {row['text_Correct']}")
    print("-" * 40)

# - pyspellchecker

In [55]:
# !pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker(language='de')
sentence = 'Dies ist ein Satz mikt einigen Fehlern.'
corrected_sentence = ' '.join([spell.correction(word) for word in sentence.split()])
print(corrected_sentence)

Dies ist ein satt mit einigen fehlen


# -  textblob


In [4]:
!pip install textblob_de

In [5]:
# !pip install textblob # the version for the English Model
# !pip install textblob_de # the version for the German Model



# from textblob import TextBlob

# def spell_correction(input_text):
#     # Create a TextBlob object from the input text
#     blob = TextBlob(input_text)

#     # Perform spell correction
#     corrected_text = blob.correct()

#     return str(corrected_text)

# # Example usage
# input_word = "teh"
# suggested_correction = spell_correction(input_word)

# if input_word != suggested_correction:
#     print(f"Suggested correction for '{input_word}': {suggested_correction}")
# else:
#     print(f"No spelling correction found for '{input_word}'.")


from textblob_de import TextBlobDE

def german_spell_correction(input_text):
    # Create a TextBlobDE object from the input text
    blob = TextBlobDE(input_text)

    # Perform spell correction
    corrected_text = blob.correct()

    return str(corrected_text)

# Example usage
input_word = "teh"
suggested_correction = german_spell_correction(input_word)

if input_word != suggested_correction:
    print(f"Suggested correction for '{input_word}': {suggested_correction}")
else:
    print(f"No spelling correction found for '{input_word}'.")



NotImplementedError: 

-    the German library with TextBlob not have coorect function

 the Textblotde not have correct fucntion till the time of this script where written 29.01.2024
 -   Properties or methods that do not yet work for German raise a NotImplementedError
 https://github.com/markuskiller/textblob-de

----------------------------------------------------------------------------------------------------------------

#   - Google 




In [57]:
!pip install googlesearch-python
from googlesearch import search

def get_google_spell_correction(query):
    try:
        # Make a Google search query
        search_query = query + " spelling correction"
        search_results = search(search_query, num_results=1, lang="en")

        # Extract the first search result (typically a spelling suggestion)
        first_result = next(search_results)

        # Return the suggested correction
        return first_result

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
input_word = "caieo"  # cario 
suggested_correction = get_google_spell_correction(input_word)

if suggested_correction:
    print(f"Suggested correction for '{input_word}': {suggested_correction}")
else:
    print(f"No spelling correction found for '{input_word}'.")


  Using cached googlesearch_python-1.2.3-py3-none-any.whl
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ---------------------------------------- 147.9/147.9 kB 2.2 MB/s eta 0:00:00
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
Suggested correction for 'caieo': https://www.spellcheck.net/cairo


#   - pyaspeller

In [58]:
# !pip install pyaspeller


In [59]:
# import pandas as pd
# from pyaspeller import YandexSpeller

# # Sample DataFrame with German text
# data = {'Text': ['Apfel is a fruut', 'Ihr Auto is blu', 'Ich liebe Schokolad']}
# df = pd.DataFrame(data)

# # Initialize the spell checker with the German language
# speller = YandexSpeller(lang='de')

# # Function to correct the spelling in a given text
# def correct_spelling(text):
#     corrected_text = speller.spelled(text)
#     return corrected_text

# # Apply the correction function to the 'Text' column of the DataFrame
# df['Corrected_Text'] = df['Text'].apply(correct_spelling)

# # Display the corrected DataFrame
# print(df)


BadArgumentError: Unsupported language

#   pyhunspell

In [60]:
!pip install hunspell

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build hunspell


  error: subprocess-exited-with-error
  
  × Building wheel for hunspell (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hunspell' extension
      creating build
      creating build\temp.win-amd64-cpython-311
      creating build\temp.win-amd64-cpython-311\Release
      "C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.38.33130\bin\HostX86\x64\cl.exe" /c /nologo /O2 /W3 /GL /DNDEBUG /MD -DHUNSPELL_STATIC -IV:/hunspell-1.3.3/src/hunspell -Ic:\A\00Master\Demo\FineTuning\Finetuning\include -IC:\Users\engin\anaconda3\include -IC:\Users\engin\anaconda3\Include "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.38.33130\include" "-IC:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Auxiliary\VS\include" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.22621.0\ucrt" "-IC:\Program Files 

In [ ]:

import hunspell

# Load the German Hunspell dictionary
german_hunspell = hunspell.Hunspell(
    r"C:\A\00Master\Source\Text Propess\Text Cleaning\de_DE_frami.dic",  # path to your German .dic file
    r"C:\A\00Master\Source\Text Propess\Text Cleaning\de_DE_frami.aff",      # path to your German .aff file
)
# "C:\A\00Master\Source\Text Propess\Text Cleaning\de_DE_frami.dic"

# Function to correct the spelling of a single word
def correct_spelling(word):
    suggestions = german_hunspell.suggest(word)
    if suggestions:
        return suggestions[0]  # Choose the first suggestion as the correction
    else:
        return word  # If no suggestion found, return the original word

# Function to correct the spelling in a text
def correct_text(text):
    words = text.split()
    corrected_words = [correct_spelling(word) for word in words]
    return ' '.join(corrected_words)



# Apply spelling correction to the DataFrame
df['corrected_text'] = df['text'].apply(correct_text)
#
df['is_changed'] = df['text'] != df['corrected_text']

# Print the corrected DataFrame
print(df)


In [ ]:
# !pip install pyspellchecker

In [ ]:
df= df_first_row

#   SpaCy
-    by using Spacy Model it seem it take so much time to excueted a Small text

In [63]:
# !pip install spacy
# !pip install contextualSpellCheck
# !python -m spacy download de_dep_news_trf
# !python -m spacy download de_core_news_sm

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.3.4-py3-none-any.whl.metadata (4.7 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached confection-0.1.4-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.16.0-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.1 MB 17.2 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.1 MB 12.2 MB/s eta 0:00:01
   ---- ----------------------------------- 1.4/12.1 MB 11.1 MB/s eta 0:00:01
   ------ --------------------------------- 2.0/12.1 MB 11.5 M

In [74]:
import pandas as pd
import tqdm as notebook_tqdm
data = {'text': [
    'Guten Moroigen! Wie geht es Ihnen?',
    'Das ist ein Beisppppieltext auf Deutsch.',
    'Die Sonne schieint heute sehr schön.',
    'Ich liebe es, Deutschhh zu sprechen!',
    'Berlin ist eine wunderbhare Stadt.'
]}

df = pd.DataFrame(data)

print(df)


                                       text
0        Guten Moroigen! Wie geht es Ihnen?
1  Das ist ein Beisppppieltext auf Deutsch.
2      Die Sonne schieint heute sehr schön.
3      Ich liebe es, Deutschhh zu sprechen!
4        Berlin ist eine wunderbhare Stadt.


In [76]:
import spacy
import contextualSpellCheck
# Load the German language model
nlp = spacy.load('de_dep_news_trf')  # or 'de_core_news_sm'

# Add the contextual spell checker to the pipeline
contextualSpellCheck.add_to_pipe(nlp)
def correct_spelling(text):
    doc = nlp(text)
    return doc._.outcome_spellCheck

# Apply spelling correction to each row in the DataFrame
df['corrected_text'] = df['text'].apply(correct_spelling)
df['is_changed'] = df['text'] != df['corrected_text']
df.head()



------------------------------------------------------------------------------------------------

In [6]:
!pip install transformers


In [7]:
import transformers
import torch

# Load a pre-trained GPT-3 model
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)

# Function to perform spelling correction
def correct_spelling(sentence):
    input_ids = tokenizer.encode(sentence, return_tensors="pt")
    with torch.no_grad():
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50)
    
    corrected_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return corrected_sentence

# Input sentence with spelling errors
input_sentence = "I hav a bok and I lik to red it."

# Perform spelling correction
corrected_sentence = correct_spelling(input_sentence)

print("Original Sentence:", input_sentence)
print("Corrected Sentence:", corrected_sentence)


c:\A\00Master\Demo\FineTuning\Finetuning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 200/200 [00:00<?, ?B/s] 
c:\A\00Master\Demo\FineTuning\Finetuning\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\engin\.cache\huggingface\hub\models--EleutherAI--gpt-neo-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate D

KeyboardInterrupt: 

NTLK 

In [ ]:
# import pandas as pd
# from spellchecker import SpellChecker



# # Initialize the SpellChecker for German
# spell_checker = SpellChecker(language='de')

# # Function to perform spell checking and get corrections for a text
# def spell_check_and_correct(text):
#     words = text.split()
#     corrections = {}

#     for word in words:
#         corrected_word = spell_checker.correction(word)
#         if corrected_word != word:
#             corrections[word] = corrected_word

#     return corrections

# # Apply spell checking and correction to the 'text' column of the DataFrame
# df_first_row['Corrections'] = df_first_row['text'].apply(spell_check_and_correct)

# # Display the DataFrame with corrections
# print(df_first_row)


--------------------------------------------------------

In [82]:
!pip install pyenchant

     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/11.9 MB 1.1 MB/s eta 0:00:11
     - -------------------------------------- 0.4/11.9 MB 3.5 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/11.9 MB 4.5 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/11.9 MB 5.1 MB/s eta 0:00:03
     ----- ---------------------------------- 1.6/11.9 MB 6.2 MB/s eta 0:00:02
     ------- -------------------------------- 2.1/11.9 MB 6.8 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/11.9 MB 7.6 MB/s eta 0:00:02
     ---------- ----------------------------- 3.2/11.9 MB 8.1 MB/s eta 0:00:02
     ------------ --------------------------- 3.7/11.9 MB 8.5 MB/s eta 0:00:01
     -------------- ------------------------- 4.3/11.9 MB 8.8 MB/s eta 0:00:01
     ---------------- ----------------------- 4.8/11.9 MB 9.1 MB/s

In [85]:
!pip install pyenchant --upgrade


In [86]:
import enchant
broker = enchant.Broker()
broker.describe()
broker.list_languages()


['en_BW',
 'en_AU',
 'en_BZ',
 'en_GB',
 'en_JM',
 'en_DK',
 'en_HK',
 'en_GH',
 'en_US',
 'en_ZA',
 'en_ZW',
 'en_SG',
 'en_NZ',
 'en_BS',
 'en_AG',
 'en_PH',
 'en_IE',
 'en_NA',
 'en_TT',
 'en_IN',
 'en_NG',
 'en_CA']

In [84]:
import enchant

# Create a German dictionary instance
dictionary = enchant.Dict("de_DE")

# Function to check the spelling of a text
def spell_check(text):
    words = text.split()
    misspelled_words = []

    for word in words:
        if not dictionary.check(word):
            misspelled_words.append(word)

    return misspelled_words

# Input German text to check for spelling errors
input_text = "Das ist ein einfaches Beispiel für die Rechtschreibprüfung mit PyEnchant."

# Perform spell checking
misspelled_words = spell_check(input_text)

if len(misspelled_words) == 0:
    print("Keine Rechtschreibfehler gefunden. (No spelling errors found.)")
else:
    print("Rechtschreibfehler gefunden: (Spelling errors found):")
    for word in misspelled_words:
        print(f"- {word}")


DictNotFoundError: Dictionary for language 'de_DE' could not be found
Please check https://pyenchant.github.io/pyenchant/ for details